In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df_ini=pd.read_csv('titanic.csv')

In [3]:
for feature in df_ini.columns[:]:
    if df_ini[feature].dtype!='object':        
        df_ini[feature]=np.where(df_ini[feature].isnull(),df_ini[feature].median(),df_ini[feature])
    else:
        df_ini[feature]=np.where(df_ini[feature].isnull(),df_ini[feature].value_counts().index[0],df_ini[feature])
df_aim=df_ini

In [4]:
def reol(feature1):
    IQR=df_aim[feature1].quantile(0.75)-df_aim[feature1].quantile(0.25)
    lbound=df_aim[feature1].quantile(0.25)-(IQR*1.5)
    ubound=df_aim[feature1].quantile(0.75)+(IQR*1.5)
    df_aim.loc[df_aim[feature1]<=lbound,feature1]=lbound
    df_aim.loc[df_aim[feature1]>=ubound,feature1]=ubound
df_aol=df_aim

In [5]:
for feature in df_aim.columns[:]:
    if df_aim[feature].dtype!='object':
        reol(feature)

In [6]:
df_aol_cont1=df_aol.select_dtypes(exclude=[object])

In [7]:
df_aol_cont2=df_aol_cont1.drop(columns='Pclass',axis=1) #Dropping categorical ordinal features

In [8]:
df_aol_cont3=df_aol_cont2.drop('Survived',axis=1)#varience threshold can be applied for independent features only

In [9]:
from sklearn.feature_selection import VarianceThreshold
var_thres=VarianceThreshold(threshold=0)
var_thres.fit(df_aol_cont3)

VarianceThreshold(threshold=0)

In [10]:
const_features=[column for column in df_aol_cont3.columns\
               if column not in df_aol_cont3.columns[var_thres.get_support()]]

In [11]:
df_aol=df_aol.drop(const_features,axis=1)#Dropping constant features

In [12]:
df_aol_cont3=df_aol_cont3.drop(columns=const_features,axis=1)

In [13]:
X=df_aol_cont3
y=df_aol_cont2['Survived']

In [14]:
def pcc(dataset,threshold):
    dropset=set()
    cor_mat=dataset.corr()
    for i in range(len(cor_mat.columns)):
        for j in range(i):
            if abs(cor_mat.iloc[i,j])>=threshold:
                cor_feat=cor_mat.columns[i]
                dropset.add(cor_feat)
    return dropset 
dropset1=pcc(X,0.85)
print("The Features to be Dropped are...")
dropset1

The Features to be Dropped are...


set()

In [15]:
df_aol=df_aol.drop(columns=dropset1,axis=1)

In [16]:
def enc(feature1):
    key_labels=df_aol[feature1].unique()
    dict={k:i for i,k in enumerate(key_labels,0)}
    df_aol[feature1]=df_aol[feature1].map(dict)

In [17]:
for feature in df_aol.columns[:]:
    if df_aol[feature].dtype=='object':
        enc(feature)
df_aec=df_aol

In [18]:
from sklearn.feature_selection import mutual_info_classif
X=df_aec.drop(columns='Survived',axis=1)
y=df_aec['Survived']
mut_inf_data=mutual_info_classif(X,y,random_state=0)
mut_inf_modif=pd.Series(mut_inf_data,index=X.columns)
mut_inf_sorted=mut_inf_modif.sort_values(ascending=False)
dict=mut_inf_sorted.to_dict()

In [19]:
droplist=[]
for key in dict:
    if dict[key]<0.001:
        droplist.append(key)

In [20]:
df_aec=df_aec.drop(columns=droplist,axis=1)

In [21]:
df_aim_cate1=df_aim.select_dtypes(include=[object])#For imbalanced data check,include all categorical features+Dep. feature

In [22]:
df_aim_cate2=df_aim[['Pclass','Survived']].copy()#include categorical ordinal feature and
df_aim_cate=pd.concat([df_aim_cate1,df_aim_cate2],axis=1)#dependent feature in the new Data frame.

In [23]:
imb_chk_list=list()

In [24]:
for feature in df_aim_cate.columns[:]:
    length=int(len(df_aim_cate[feature].unique()))      
    if length <= 5:
        imb_chk_list.append(feature)

In [25]:
drop_list=[]#Add the independent features with single category, imbalanced data in this list(Difference of values<0.05 and >0.95)

In [26]:
df_aec=df_aec.drop(drop_list,axis=1)

In [27]:
original_list=['Survived','Pclass','Sex','SibSp','Cabin','Embarked']
log_list=['Fare']
expo_list=[]
sqroot_list=['PassengerId','Name','Ticket']
boxcox_list=[]

In [28]:
for feature in log_list:
    df_aec[feature]=np.log1p(df_aec[feature])

In [29]:
for feature in sqroot_list:
    df_aec[feature]=df_aec[feature]**(1/2)

In [30]:
df_atr=df_aec

In [31]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaled=scaler.fit_transform(df_atr)

In [32]:
df_ast=pd.DataFrame(scaled,columns=df_atr.columns)

In [33]:
df_fin=df_ast

In [34]:
X=df_fin.drop(columns='Survived',axis=1)
y=df_fin['Survived']

In [35]:
from sklearn.model_selection import train_test_split

In [36]:
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.7,random_state=0)

In [37]:
from sklearn.linear_model import LogisticRegression
classifier=LogisticRegression()
classifier.fit(X_train,y_train)
prediction=classifier.predict(X_test)

In [38]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(confusion_matrix(y_test,prediction))
print("Accuracy Score :",accuracy_score(y_test,prediction))
print(classification_report(y_test,prediction))

[[142  26]
 [ 27  73]]
Accuracy Score : 0.8022388059701493
              precision    recall  f1-score   support

         0.0       0.84      0.85      0.84       168
         1.0       0.74      0.73      0.73       100

    accuracy                           0.80       268
   macro avg       0.79      0.79      0.79       268
weighted avg       0.80      0.80      0.80       268



In [39]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
classifier=LogisticRegression()
classifier.get_params().keys()

dict_keys(['C', 'class_weight', 'dual', 'fit_intercept', 'intercept_scaling', 'l1_ratio', 'max_iter', 'multi_class', 'n_jobs', 'penalty', 'random_state', 'solver', 'tol', 'verbose', 'warm_start'])

In [40]:
param_grid={'penalty':['l2','l1'],
            'C':np.logspace(4,6,10),
            'solver':['newton-cg','lbfgs','sags','sag','liblinear']}            

In [41]:
grid_search=RandomizedSearchCV(estimator=classifier,param_distributions=param_grid,cv=10,
                         verbose=2,random_state=100,n_jobs=-1)

In [42]:
grid_search.fit(X_train,y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    4.1s finished


RandomizedSearchCV(cv=10, estimator=LogisticRegression(), n_jobs=-1,
                   param_distributions={'C': array([  10000.        ,   16681.005372  ,   27825.59402207,
         46415.88833613,   77426.36826811,  129154.96650149,
        215443.46900319,  359381.36638046,  599484.25031894,
       1000000.        ]),
                                        'penalty': ['l2', 'l1'],
                                        'solver': ['newton-cg', 'lbfgs', 'sags',
                                                   'sag', 'liblinear']},
                   random_state=100, verbose=2)

In [43]:
best_grid=grid_search.best_estimator_
best_grid

LogisticRegression(C=77426.36826811278)

In [44]:
prediction=best_grid.predict(X_test)

In [45]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(confusion_matrix(y_test,prediction))
print("Accuracy Score :",accuracy_score(y_test,prediction))
print(classification_report(y_test,prediction))

[[141  27]
 [ 26  74]]
Accuracy Score : 0.8022388059701493
              precision    recall  f1-score   support

         0.0       0.84      0.84      0.84       168
         1.0       0.73      0.74      0.74       100

    accuracy                           0.80       268
   macro avg       0.79      0.79      0.79       268
weighted avg       0.80      0.80      0.80       268



In [46]:
# pip install optuna
import optuna
from sklearn.model_selection import cross_val_score
def objective(trial):
    penalty=trial.suggest_categorical('penalty',['l2','l1'])
    solver=trial.suggest_categorical('solver',['newton-cg','lbfgs','sags','sag','liblinea'])
    C=trial.suggest_uniform('C',0.01,10)
    classifier=LogisticRegression(penalty=penalty,solver=solver,C=C)
    score=cross_val_score(classifier,X_train,y_train,n_jobs=-1,cv=3).mean()
    return score

In [47]:
study=optuna.create_study(direction='maximize')
study.optimize(objective,n_trials=100)

[I 2021-07-27 22:59:52,226] A new study created in memory with name: no-name-f2db3268-aef2-40ce-a290-7cb2d245dbdd
[W 2021-07-27 22:59:52,247] Trial 0 failed, because the objective function returned nan.
[I 2021-07-27 22:59:52,290] Trial 1 finished with value: 0.7977904744209092 and parameters: {'penalty': 'l2', 'solver': 'lbfgs', 'C': 9.526236364574254}. Best is trial 1 with value: 0.7977904744209092.
[W 2021-07-27 22:59:52,310] Trial 2 failed, because the objective function returned nan.
[I 2021-07-27 22:59:52,357] Trial 3 finished with value: 0.7977904744209092 and parameters: {'penalty': 'l2', 'solver': 'lbfgs', 'C': 8.772351448597735}. Best is trial 1 with value: 0.7977904744209092.
[I 2021-07-27 22:59:52,396] Trial 4 finished with value: 0.7946085717824848 and parameters: {'penalty': 'l2', 'solver': 'lbfgs', 'C': 0.16491283230442716}. Best is trial 1 with value: 0.7977904744209092.
[I 2021-07-27 22:59:52,429] Trial 5 finished with value: 0.7913724761550848 and parameters: {'penalt

[W 2021-07-27 22:59:54,077] Trial 68 failed, because the objective function returned nan.
[W 2021-07-27 22:59:54,103] Trial 69 failed, because the objective function returned nan.
[W 2021-07-27 22:59:54,126] Trial 70 failed, because the objective function returned nan.
[W 2021-07-27 22:59:54,150] Trial 71 failed, because the objective function returned nan.
[W 2021-07-27 22:59:54,175] Trial 72 failed, because the objective function returned nan.
[W 2021-07-27 22:59:54,198] Trial 73 failed, because the objective function returned nan.
[W 2021-07-27 22:59:54,221] Trial 74 failed, because the objective function returned nan.
[W 2021-07-27 22:59:54,244] Trial 75 failed, because the objective function returned nan.
[W 2021-07-27 22:59:54,266] Trial 76 failed, because the objective function returned nan.
[W 2021-07-27 22:59:54,290] Trial 77 failed, because the objective function returned nan.
[W 2021-07-27 22:59:54,315] Trial 78 failed, because the objective function returned nan.
[W 2021-07

In [48]:
best=study.best_params

In [49]:
best

{'penalty': 'l2', 'solver': 'lbfgs', 'C': 9.526236364574254}

In [50]:
classifier=LogisticRegression(penalty=best['penalty'],C=best['C'],solver=best['solver'])
classifier.fit(X_train,y_train)
prediction=classifier.predict(X_test)

In [51]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(confusion_matrix(y_test,prediction))
print("Accuracy Score :",accuracy_score(y_test,prediction))
print(classification_report(y_test,prediction))

[[142  26]
 [ 26  74]]
Accuracy Score : 0.8059701492537313
              precision    recall  f1-score   support

         0.0       0.85      0.85      0.85       168
         1.0       0.74      0.74      0.74       100

    accuracy                           0.81       268
   macro avg       0.79      0.79      0.79       268
weighted avg       0.81      0.81      0.81       268



In [52]:
# pip install hyperopt
from hyperopt import hp,fmin,tpe,STATUS_OK,Trials
space={'penalty':hp.choice('penalty',['l2','l1']),
       'solver':hp.choice('solver',['newton-cg','lbfgs','sags','sag','liblinea']),
       'C':hp.uniform('C',0.01,10)}

In [53]:
def objective(space):
    classifier=LogisticRegression(penalty=space['penalty'],solver=space['solver'],
               C=space['C'])
    score=cross_val_score(classifier,X_train,y_train,n_jobs=-1,cv=5).mean()
    return{'loss':-score,'status':STATUS_OK}

In [54]:
trials=Trials()
best=fmin(fn=objective,space=space,algo=tpe.suggest,max_evals=80,trials=trials)
best

100%|███████████████████████████████████████████████| 80/80 [00:02<00:00, 36.36trial/s, best loss: -0.7913806451612904]


{'C': 0.47376354316499514, 'penalty': 0, 'solver': 3}

In [55]:
pen={0:'l2',1:'l1'}
sol={0:'newton-cg',1:'lbfgs',2:'sags',3:'sag',4:'liblinea'}

In [56]:
classifier=LogisticRegression(penalty=pen[best['penalty']],
            solver=sol[best['solver']],C=best['C'])
classifier.fit(X_train,y_train)
prediction=classifier.predict(X_test)

In [57]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(confusion_matrix(y_test,prediction))
print("Accuracy Score :",accuracy_score(y_test,prediction))
print(classification_report(y_test,prediction))

[[142  26]
 [ 27  73]]
Accuracy Score : 0.8022388059701493
              precision    recall  f1-score   support

         0.0       0.84      0.85      0.84       168
         1.0       0.74      0.73      0.73       100

    accuracy                           0.80       268
   macro avg       0.79      0.79      0.79       268
weighted avg       0.80      0.80      0.80       268

